### <h2 style="font-size: 40px;">Состояние пациента по отзыву на лекарство</h2>

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import torch

In [ ]:
#!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
#!unzip drugsCom_raw.zip -d ./datasets/
#!rm drugsCom_raw.zip

--2025-06-09 22:52:23--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Распознаётся archive.ics.uci.edu (archive.ics.uci.edu)… 128.195.10.252
Подключение к archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: нет данных
Сохранение в: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [ <=>                ]  41,00M   191KB/s    за 3m 40s  

2025-06-09 22:56:05 (191 KB/s) - ‘drugsCom_raw.zip’ сохранён [42989872]

Archive:  drugsCom_raw.zip
  inflating: /home/serguntsov/datasets/drugsComTest_raw.tsv  
  inflating: /home/serguntsov/datasets/drugsComTrain_raw.tsv  
